In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [9]:
from huggingface_hub import HfFileSystem
from tqdm import tqdm
from glob import glob
import os
import pandas as pd
import re
import polars as pl
import json
import mp

def clean(string):
    string = re.sub('[^a-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [3]:
keywords = {
    'malay', 'malaysia', 'melayu', 'bursa'
}

In [4]:
fs = HfFileSystem()

In [5]:
files = fs.ls("datasets/HuggingFaceTB/smollm-corpus/cosmopedia-v2", detail=False)
files = sorted(files)

In [6]:
def loop(files):
    files, _ = files
    fs = HfFileSystem()
    for f in files:
        f_only = os.path.split(f)[1]
        f_only_new = f'{f_only}.jsonl'
        f_only_done = f'{f_only}.done'
        if os.path.exists(f_only_done):
            continue
        
        try:
            if not os.path.exists(f_only):
                fs.get(f, f_only)
            df = pl.read_parquet(f_only)
        except:
            fs.get(f, f_only)
            df = pl.read_parquet(f_only)
        
        with open(f_only_new, 'w') as fopen:
            for i in tqdm(range(len(df))):
                row = df[i].to_dict(as_series = False)
                for k in row.keys():
                    row[k] = row[k][0]
                
                a = set(clean(row['text']).split()) & keywords
                if len(a):
                    fopen.write(f'{json.dumps(row)}\n')
        
        with open(f_only_done, 'w') as fopen:
            fopen.write('done')
        
        os.remove(f_only)
        del df

In [7]:
loop((files[:1], 0))

In [8]:
mp.multiprocessing(files, loop, cores = 6, returned = False)

In [13]:
files = glob('train-*.parquet.jsonl')

with open('cosmopedia-v2.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                fopen_l.write(f'{json.dumps(l)}\n')

In [14]:
!wc -l cosmopedia-v2.jsonl

85992 cosmopedia-v2.jsonl


In [15]:
!ls -lh cosmopedia-v2.jsonl

-rw-rw-r-- 1 husein husein 492M Ogos  7 13:23 cosmopedia-v2.jsonl


In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="cosmopedia-v2.jsonl",
    path_in_repo="cosmopedia-v2.jsonl",
    repo_id="mesolitica/smollm-corpus-filter-malaysian-context",
    repo_type="dataset",
)

cosmopedia-v2.jsonl:   0%|          | 0.00/515M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/smollm-corpus-filter-malaysian-context/commit/68d54e29ac37556266654d34d208c6a27a64e496', commit_message='Upload cosmopedia-v2.jsonl with huggingface_hub', commit_description='', oid='68d54e29ac37556266654d34d208c6a27a64e496', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
!rm train-*.parquet.done